# Run Foundry Agent with Foundry IQ Knowledge Base

This notebook runs the repository flow with these guarantees:
- The assistant is created on **Microsoft Foundry** using the **Prompt Agents API**.
- Retrieval uses the **Foundry IQ knowledge base** (`sharepoint-kb`) via its MCP connection.
- The `MCPTool` with `project_connection_id` authenticates via **Project Managed Identity**.

This notebook can run on a fresh repo (not yet provisioned):
- Run Cell 2 to authenticate and provision infrastructure (`azd auth login` + `azd up`).
- Then continue with the remaining cells.

In [ ]:
# One-time infra setup (can take several minutes)
!azd auth login
!azd up

In [ ]:
%pip install -q "azure-identity" "azure-ai-projects>=2.0.0b1" "openai"

In [ ]:
import subprocess

def get_azd_env() -> dict:
    try:
        output = subprocess.check_output(["azd", "env", "get-values"], text=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as exc:
        raise RuntimeError(
            "Could not read azd environment values. Run Cell 2 first to provision (azd auth login + azd up)."
        ) from exc

    values = {}
    for line in output.splitlines():
        if "=" not in line:
            continue
        key, value = line.split("=", 1)
        values[key.strip()] = value.strip().strip("\"").strip("'")
    return values

env = get_azd_env()

required = ["PROJECT_ENDPOINT", "MODEL_DEPLOYMENT"]
missing = [key for key in required if not env.get(key)]
if missing:
    raise RuntimeError(
        f"Missing required azd outputs: {missing}. Run Cell 2 first, then rerun this cell."
    )

PROJECT_ENDPOINT = env["PROJECT_ENDPOINT"]
MODEL_DEPLOYMENT = env["MODEL_DEPLOYMENT"]

print("PROJECT_ENDPOINT:", PROJECT_ENDPOINT)
print("MODEL_DEPLOYMENT:", MODEL_DEPLOYMENT)

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

credential = DefaultAzureCredential()

project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=credential,
)

# Look up the KB MCP connection created by postdeploy.sh
KB_CONNECTION_NAME = "kb-sharepoint-kb"
kb_conn = project_client.connections.get(KB_CONNECTION_NAME)

# The connection ID is the full ARM resource path — needed by MCPTool
KB_CONNECTION_ID = kb_conn.id
KB_MCP_URL = kb_conn.target  # The KB MCP endpoint URL

print("KB connection:", kb_conn.name)
print("KB MCP URL:", KB_MCP_URL)
print("Connection ID:", KB_CONNECTION_ID)

In [ ]:
from azure.ai.projects.models import MCPTool, PromptAgentDefinition, Tool

mcp_tool = MCPTool(
    server_label="kb_sharepoint_kb",
    server_url=KB_MCP_URL,
    require_approval="never",
    project_connection_id=KB_CONNECTION_ID,
)

print("MCPTool ready")
print("  Server label:", mcp_tool.server_label)
print("  Server URL:", KB_MCP_URL)
print("  Connection ID:", KB_CONNECTION_ID)

In [ ]:
AGENT_NAME = "foundryiq-sharepoint-agent"
INSTRUCTIONS = (
    "You are an HR assistant that answers questions using the knowledge base. "
    "Use the available tools to search for relevant information before answering. "
    "Answer in the same language as the question.\n\n"
    "RULES:\n"
    "1) Base your answer ONLY on information retrieved from the knowledge base.\n"
    "2) Do NOT use inline citation markers like 【4:0†source】 in your answer text.\n"
    "3) Use ONLY doc_url values returned by the tool — NEVER invent or modify URLs.\n"
    "4) Only include URLs that start with https:// — skip relative paths or Graph API paths.\n"
    "5) If a snippet starts with [Page X], extract the page number for citations.\n\n"
    "OUTPUT FORMAT — you MUST follow this exact structure:\n\n"
    "## Réponse\n"
    "[Your detailed answer here, organized with numbered points or bullet points as appropriate.]\n\n"
    "## Sources\n"
    "| Document | Pages | Lien |\n"
    "| --- | --- | --- |\n"
    "| [Document title] | [Page numbers used, e.g. 1, 3, 5] | [Clickable link](doc_url) |\n"
)

tools: list[Tool] = [mcp_tool]

agent = project_client.agents.create_version(
    agent_name=AGENT_NAME,
    definition=PromptAgentDefinition(
        model=MODEL_DEPLOYMENT,
        instructions=INSTRUCTIONS,
        tools=tools,
    ),
    description="HR SharePoint agent using Foundry IQ KB via MCP",
)

print(f"Agent created: id={agent.id}, name={agent.name}, version={agent.version}")

In [ ]:
from openai.types.responses.response_input_param import McpApprovalResponse, ResponseInputParam

QUESTION = "Quelles sont les conditions de retour au travail après un congé maternité ?"
print(f"Question: {QUESTION}\n")

openai_client = project_client.get_openai_client()

# Create a conversation thread
conversation = openai_client.conversations.create()
print(f"Conversation: {conversation.id}")

# Send the question to the agent
response = openai_client.responses.create(
    conversation=conversation.id,
    input=QUESTION,
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)

# Auto-approve any MCP tool approval requests
input_list: ResponseInputParam = []
for item in response.output:
    if item.type == "mcp_approval_request":
        print(f"Approving MCP request: {item.id} (server: {item.server_label})")
        if item.id:
            input_list.append(
                McpApprovalResponse(
                    type="mcp_approval_response",
                    approve=True,
                    approval_request_id=item.id,
                )
            )

# If there were approval requests, send the approvals to continue
if input_list:
    response = openai_client.responses.create(
        input=input_list,
        previous_response_id=response.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
    )

# Print the final response
print("\n--- Agent Response ---\n")
print(response.output_text)

# Cleanup
# project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
print("\n--- Agent deleted ---")

## Notes

- Uses `AIProjectClient` (azure-ai-projects ≥ 2.0.0b1) with the **Prompt Agents API**.
- Uses `MCPTool` with `project_connection_id` pointing to the `kb-sharepoint-kb` connection.
- The connection authenticates to the KB MCP server using **Project Managed Identity** — no tokens in code.
- Agent is created with `create_version()` + `PromptAgentDefinition`, and invoked via `openai_client.responses.create()`.
- The Foundry IQ knowledge base (`sharepoint-kb`) is queried exclusively via its MCP endpoint.
- **No `AzureAISearchTool`, no direct AI Search calls, no raw REST.**
- If connection lookup fails, verify the connection exists: `azd hooks run postdeploy`.